In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import StratifiedKFold, GridSearchCV
import xgboost
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_log_error,r2_score,accuracy_score,confusion_matrix

import datetime
from sklearn import preprocessing
sns.set()

train = pd.read_csv('/kaggle/input/dmwmini/train.csv')
test = pd.read_csv('/kaggle/input/dmwmini/test.csv')

train.info()

In [ ]:
train.head()

In [ ]:
from datetime import datetime
train['date'] = pd.to_datetime(train['date'],format='%m/%d/%Y')
# train['month'] = train['date'].dt.month
# train['dayofweek'] = train['date'].dt.dayofweek
# train['year'] = train['date'].dt.year
# train['day'] = train['date'].dt.day
# train['dayofyear'] = train['date'].dt.dayofyear
# train['weekofyear'] = train['date'].dt.weekofyear

train.drop(['date'], axis=1, inplace=True) 
train.head()

In [ ]:
train.isnull().sum()

In [ ]:
g = sns.FacetGrid(train, col='game_result')
g.map(plt.hist, 'Elo', bins=20)

g = sns.FacetGrid(train, col='game_result')
g.map(plt.hist, 'opp_Elo', bins=20)

g = sns.FacetGrid(train, col='game_result')
g.map(plt.hist, 'total_crowd', bins=20)



In [ ]:
for index,row in train.iterrows():
    if(row['bet_ratio']<0.13 or row['bet_ratio']>0.97):
        train.drop(index,inplace=True)  
    elif(row['opp_Elo']<1200 and row['opp_Elo']>1750):
        train.drop(index,inplace=True)  
    elif(row['Elo']<1175 and row['Elo']>1780):
        train.drop(index,inplace=True)
    elif(row['win_equivalent']<14 or row['win_equivalent']>68):
        train.drop(index,inplace=True)   

In [ ]:
# train = pd.concat([train,pd.get_dummies(train['team_id'], prefix='team_id',dummy_na=False)],axis=1)
# train = pd.concat([train,pd.get_dummies(train['opp_team_id'], prefix='opp_team_id',dummy_na=False)],axis=1)

train['team_id'] = train['team_id'].str.extract('(\\d+)', expand=True)
train['opp_team_id'] = train['opp_team_id'].str.extract('(\\d+)', expand=True)

train['team_id']=train['team_id'].astype(int)
train['opp_team_id']=train['opp_team_id'].astype(int)

In [ ]:
train['bet_ratio'].describe()


In [ ]:
# train['Elo'].fillna(train.groupby('team_id')['Elo'].transform('median'), inplace=True)
# train['opp_Elo'].fillna(train.groupby('opp_team_id')['opp_Elo'].transform('median'), inplace=True)
# train['win_equivalent'].fillna(train.groupby('season_end')['win_equivalent'].transform('median'), inplace=True)

train["temp"] = train["team_id"].map(str) + train["season_end"].map(str)
train["temp1"] = train["opp_team_id"].map(str) + train["season_end"].map(str)

train['Elo'].fillna(train.groupby('temp')['Elo'].transform('median'), inplace=True)
train['opp_Elo'].fillna(train.groupby('temp1')['opp_Elo'].transform('median'), inplace=True)
train['win_equivalent'].fillna(train.groupby('season_end')['win_equivalent'].transform('median'), inplace=True)

train = train.dropna(axis = 0, how ='any')

train.drop(['temp'], axis=1, inplace=True) 
train.drop(['temp1'], axis=1, inplace=True) 


In [ ]:
train['diff']=train.Elo-train.opp_Elo
train.head()


In [ ]:


train.drop(['Id'], axis=1, inplace=True)
train.drop(['game_seq'], axis=1, inplace=True)  
# train.drop(['season_end'], axis=1, inplace=True)        
# train.drop(['season_game_seq'], axis=1, inplace=True) 
# train.drop(['playoff'], axis=1, inplace=True)
train.drop(['team_id'], axis=1, inplace=True) 
# train.drop(['Elo'], axis=1, inplace=True)
train.drop(['opp_team_id'], axis=1, inplace=True)
# train.drop(['opp_Elo'], axis=1, inplace=True) 
# train.drop(['win_equivalent'], axis=1, inplace=True) 
# train.drop(['bet_ratio'], axis=1, inplace=True) 
# train.drop(['home_crowd'], axis=1, inplace=True) 
train.drop(['opp_crowd'], axis=1, inplace=True) 
# train.drop(['total_crowd'], axis=1, inplace=True) 

train.head()

In [ ]:
labelencoder = LabelEncoder()
train['playoff'] = labelencoder.fit_transform(train['playoff'])

In [ ]:
train.head()

In [ ]:
train.corr()

In [ ]:
y = train["game_result"]
X = train.drop(["game_result"],axis=1)

# scaler=MinMaxScaler()
# X = scaler.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split 

x_train , x_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
# scaler=StandardScaler()
# X_train = scaler.fit_transform(x_train)
# X_test = scaler.transform(x_test)
X_train = x_train
X_test = x_test

In [ ]:
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(bootstrap=True, n_estimators= 100)
rfc.fit(X_train,y_train)

y_pred= rfc.predict(X_test)
r2 = r2_score(y_test, y_pred)
acc= accuracy_score(y_test,y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
lmse = mean_squared_log_error(y_test, y_pred)
rmse = np.sqrt(mse)
lrmse = np.sqrt(lmse)
print("Random Forest Classifier :")
print("R2 score:", r2)
print("ACC:", acc)
print("MAE:", mae)
print("MSE:", mse)
print("LMSE:", lmse)
print("RMSE:", rmse)
print("LRMSE:", lrmse)
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Win','Lose'],
                      title='Confusion matrix for random forest classifier')

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knc = KNeighborsClassifier(n_neighbors=5)
knc.fit(X_train, y_train)

y_pred= knc.predict(X_test)
r2 = r2_score(y_test, y_pred)
acc= accuracy_score(y_test,y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
lmse = mean_squared_log_error(y_test, y_pred)
rmse = np.sqrt(mse)
lrmse = np.sqrt(lmse)
print("KN neighbors Classifier :")
print("R2 score:", r2)
print("ACC:", acc)
print("MAE:", mae)
print("MSE:", mse)
print("LMSE:", lmse)
print("RMSE:", rmse)
print("LRMSE:", lrmse)
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Win','Lose'],
                      title='Confusion matrix for K Neighbours classifier')

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)

y_pred= lr.predict(X_test)
r2 = r2_score(y_test, y_pred)
acc= accuracy_score(y_test,y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
lmse = mean_squared_log_error(y_test, y_pred)
rmse = np.sqrt(mse)
lrmse = np.sqrt(lmse)
print("Logistic Regression :")
print("R2 score:", r2)
print("ACC:", acc)
print("MAE:", mae)
print("MSE:", mse)
print("LMSE:", lmse)
print("RMSE:", rmse)
print("LRMSE:", lrmse)
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Win','Lose'],
                      title='Confusion matrix for logistic regression')

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)

y_pred= lr.predict(X_test)
r2 = r2_score(y_test, y_pred)
acc= accuracy_score(y_test,y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
lmse = mean_squared_log_error(y_test, y_pred)
rmse = np.sqrt(mse)
lrmse = np.sqrt(lmse)
print("Logistic Regression :")
print("R2 score:", r2)
print("ACC:", acc)
print("MAE:", mae)
print("MSE:", mse)
print("LMSE:", lmse)
print("RMSE:", rmse)
print("LRMSE:", lrmse)
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Win','Lose'],
                      title='Confusion matrix for logistic regression')

In [ ]:
# from sklearn import svm
# svc = svm.SVC(kernel='linear') 
# svc.fit(X_train, y_train)

# y_pred= svc.predict(X_test)
# r2 = r2_score(y_test, y_pred)
# acc= accuracy_score(y_test,y_pred)
# mae = mean_absolute_error(y_test, y_pred)
# mse = mean_squared_error(y_test, y_pred)
# lmse = mean_squared_log_error(y_test, y_pred)
# rmse = np.sqrt(mse)
# lrmse = np.sqrt(lmse)
# print("Support Vector Classifier :")
# print("R2 score:", r2)
# print("ACC:", acc)
# print("MAE:", mae)
# print("MSE:", mse)
# print("LMSE:", lmse)
# print("RMSE:", rmse)
# print("LRMSE:", lrmse)

In [ ]:
xgc = xgboost.XGBClassifier(random_state = 1, max_depth = 15,n_estimators = 200, min_samples_split = 2, min_samples_leaf = 1)
xgc=xgc.fit(X_train, y_train)

y_pred= xgc.predict(X_test)
r2 = r2_score(y_test, y_pred)
acc= accuracy_score(y_test,y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
lmse = mean_squared_log_error(y_test, y_pred)
rmse = np.sqrt(mse)
lrmse = np.sqrt(lmse)
print("XFB Classifier :")
print("R2 score:", r2)
print("ACC:", acc)
print("MAE:", mae)
print("MSE:", mse)
print("LMSE:", lmse)
print("RMSE:", rmse)
print("LRMSE:", lrmse)
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Win','Lose'],
                      title='Confusion matrix for XGBoost classifier')

In [ ]:
# feature_importances = pd.DataFrame(lr.feature_importances_,
#                                    index = X_train.columns,
#                                     columns=['importance']).sort_values('importance',ascending=False)
# feature_importances

In [ ]:
run_gs = False # make true for grid search

if run_gs:
    parameter_grid = {
                 'max_depth' : [4, 6, 8],
                 'n_estimators': [50, 10, 20],
                 'max_features': ['sqrt', 'auto', 'log2'],
                 'min_samples_split': [2, 3, 10],
                 'min_samples_leaf': [1, 3, 10],
                 'bootstrap': [True, False],
                 }
    forest = RandomForestClassifier()
    cross_validation = StratifiedKFold(n_splits=1)

    grid_search = GridSearchCV(forest,
                               scoring='accuracy',
                               param_grid=parameter_grid,
                               cv=cross_validation,
                               verbose=1,
                               n_jobs=-1
                              )

    grid_search.fit(X_train, y_train)
    model = grid_search
    parameters = grid_search.best_params_

    print('Best score: {}'.format(grid_search.best_score_))
    print('Best parameters: {}'.format(grid_search.best_params_))
    


In [ ]:
from datetime import datetime
test['date'] = pd.to_datetime(test['date'],format='%m/%d/%Y')
# test['month'] = test['date'].dt.month
# test['dayofweek'] = test['date'].dt.dayofweek
# test['year'] = test['date'].dt.year
# test['day'] = test['date'].dt.day
# test['dayofyear'] = test['date'].dt.dayofyear
# test['weekofyear'] = test['date'].dt.weekofyear

test.drop(['date'], axis=1, inplace=True) 

In [ ]:
# test = pd.concat([test,pd.get_dummies(test['team_id'], prefix='team_id',dummy_na=False)],axis=1)
# test = pd.concat([test,pd.get_dummies(test['opp_team_id'], prefix='opp_team_id',dummy_na=False)],axis=1)

In [ ]:
test['team_id'] = test['team_id'].str.extract('(\\d+)', expand=True)
test['opp_team_id'] = test['opp_team_id'].str.extract('(\\d+)', expand=True)

test['team_id']=test['team_id'].astype(int)
test['opp_team_id']=test['opp_team_id'].astype(int)

In [ ]:
# test['Elo'].fillna(test.groupby('team_id')['Elo'].transform('mean'), inplace=True)
# test['opp_Elo'].fillna(test.groupby('opp_team_id')['opp_Elo'].transform('mean'), inplace=True)
# test['win_equivalent'].fillna(test.groupby('season_end')['win_equivalent'].transform('mean'), inplace=True)

test["temp"] = test["team_id"].map(str) + test["season_end"].map(str)
test["temp1"] = test["opp_team_id"].map(str) + test["season_end"].map(str)

test['Elo'].fillna(test.groupby('temp')['Elo'].transform('median'), inplace=True)
test['opp_Elo'].fillna(test.groupby('temp1')['opp_Elo'].transform('median'), inplace=True)
test['win_equivalent'].fillna(test.groupby('season_end')['win_equivalent'].transform('median'), inplace=True)

test['Elo'].fillna(test.groupby('team_id')['Elo'].transform('mean'), inplace=True)
test['opp_Elo'].fillna(test.groupby('opp_team_id')['opp_Elo'].transform('mean'), inplace=True)

test = test.dropna(axis = 0, how ='any')

test.drop(['temp'], axis=1, inplace=True) 
test.drop(['temp1'], axis=1, inplace=True) 

In [ ]:
# for index,row in test.iterrows():
    
#     if(test.loc[index,'bet_ratio']<0.5):
#         test.at[index,'Bet']=-1 
#     if(test.loc[index,'bet_ratio']>0.8):
#         test.at[index,'Bet']=1
#     if(test.loc[index,'bet_ratio']>=0.5 and test.loc[index,'bet_ratio']<=0.8):
#         test.at[index,'Bet']=0
#     if(test.loc[index,'Elo']<1350):
#         test.at[index,'elo1']=-1 
#     if(test.loc[index,'Elo']>1645):
#         test.at[index,'elo1']=1
#     if(test.loc[index,'Elo']>=1350 and test.loc[index,'Elo']<=1645):
#         test.at[index,'elo1']=0
#     if(test.loc[index,'opp_Elo']<1355):
#         test.at[index,'elo2']=-1 
#     if(test.loc[index,'opp_Elo']>1660):
#         test.at[index,'elo2']=1
#     if(test.loc[index,'opp_Elo']>=1355 and test.loc[index,'opp_Elo']<=1660):
#         test.at[index,'elo2']=0


In [ ]:
test['diff']=test.Elo-test.opp_Elo


In [ ]:
Id = test['Id']

test.drop(['Id'], axis=1, inplace=True)
test.drop(['game_seq'], axis=1, inplace=True)  
# test.drop(['season_end'], axis=1, inplace=True)        
# test.drop(['season_game_seq'], axis=1, inplace=True) 
# test.drop(['playoff'], axis=1, inplace=True)
test.drop(['team_id'], axis=1, inplace=True) 
# test.drop(['Elo'], axis=1, inplace=True)
test.drop(['opp_team_id'], axis=1, inplace=True)
# test.drop(['opp_Elo'], axis=1, inplace=True) 
# test.drop(['win_equivalent'], axis=1, inplace=True) 
# test.drop(['bet_ratio'], axis=1, inplace=True) 
# test.drop(['home_crowd'], axis=1, inplace=True) 
test.drop(['opp_crowd'], axis=1, inplace=True) 
# test.drop(['total_crowd'], axis=1, inplace=True) 

In [ ]:
labelencoder = LabelEncoder()
test['playoff'] = labelencoder.fit_transform(test['playoff'])
# test['team_id'] = labelencoder.fit_transform(test['team_id'])
# test['opp_team_id'] = labelencoder.fit_transform(test['opp_team_id'])

In [ ]:
test.head()

In [ ]:
# scaler=StandardScaler()
# test = scaler.transform(test)

In [ ]:
prediction_test=xgc.predict(test)

In [ ]:
prediction_test=prediction_test.astype(int)

In [ ]:
my_solution=pd.DataFrame(prediction_test,Id,columns=["game_result"])
print(my_solution.shape)

In [ ]:
my_solution.to_csv("sub123.csv",index_label=["Id"])

In [ ]:
my_solution['game_result'].value_counts()